# Importando as Bibliotecas

In [ ]:
import ranking
import salvar_resultados as sr

import pandas as pd
import numpy as np
import string

import time

# Constantes

In [ ]:
COLUNAS = ("indice", "titulo_sa", "ean", "categoria")
LISTA_CATEGORIAS = ("celulares", "notebooks", "geladeiras", "fogoes", "tvs")

# Lendo os Datasets

In [ ]:
#arquivos = ['ale_1_1', 'ale_5_1', 'hn_1_1', 'hn_5_1']
#arquivos = ['ale_1_1']
#arquivos = ['ale_5_1']
#arquivos = ['hn_1_1']
arquivos = ['hn_5_1']

lista_df_treino = []
lista_df_teste = []
for arquivo in arquivos:

    df_treino = pd.read_csv(f"Dados/Datasets/Treino/{arquivo}_treino.csv", dtype = {'ean_1': str, 'ean_2': str})
    df_teste = pd.read_csv(f"Dados/Datasets/Teste/{arquivo}_teste.csv", dtype = {'ean_1': str, 'ean_2': str})
    lista_df_treino.append(df_treino)
    lista_df_teste.append(df_teste)

# Funções

In [ ]:
def criar_dicionario(indice, titulo_sa, ean, categoria):
    
    return {
            'indice' : indice, 'titulo_sa' : titulo_sa, 'ean' : ean, 'categoria' : categoria
           }


def criar_df_match(df_concat, ean_repetido):

    df_matches = pd.DataFrame(columns = COLUNAS)
    for ean in ean_repetido:

        # pega o indice da primeira linha com aquele EAN
        filtro = (df_concat['ean'] == ean)
        indice = next(iter(filtro.index[filtro]))

        dicionario = criar_dicionario(
                                    indice = indice,
                                    titulo_sa = df_concat.loc[indice]['titulo_sa'],
                                    ean = df_concat.loc[indice]['ean'],
                                    categoria = df_concat.loc[indice]['categoria']
                                    )

        df_matches = df_matches.append(dicionario, ignore_index = True)

    df_matches.sort_values('indice', inplace = True)
    df_matches.reset_index(drop = True, inplace = True)

    return df_matches


def salvar_resultado(nome, df):

    df.to_csv(f'Dados/Resultados/Cos_Rankeado/{nome}_métricas.csv', index = False)


def salvar_resultado_categoria(nome, df):

    for categoria in LISTA_CATEGORIAS:

        df[df["categoria"] == categoria].to_csv(f'Dados/Resultados/Cos_Rankeado/{categoria}/{nome}_métricas.csv', index = False)

# BoW

In [ ]:
flag_cos = True
if flag_cos == True:

    dir_metricas = f"Dados/Resultados/Ranqueado/Cos"

    for nome, df_teste in zip(arquivos, lista_df_teste):

        # remoção de pontuação e acentos
        ranking.fazer_pre_processamento(df_teste)

        df_concat, df_matches = ranking.criar_df_teste(df_teste)

        # calculando a quantidade máxima de palavras no título
        tam_max = ranking.calcular_tam_max(df_concat['titulo_sa'])

        inicio_tempo = time.time()
        # calculando o BoW de cada título
        cv, titulo_bow = ranking.formatar_entrada_bow(df_concat['titulo_sa'], mf = tam_max)
        # calculando a distância entre os vetores (apenas linhas com match)
        resultado = ranking.calcular_dis_2_vetores_cond(titulo_bow, df_matches)
        final_tempo = time.time()
        tempo = final_tempo - inicio_tempo

        # adicionando o tempo que demorou para o algoritmo rodar
        ranking.calcular_tempo(df_matches, inicio_tempo, final_tempo)

        # calcular acuracia_k e match rank
        indices, valores = ranking.calcular_metricas(df_matches, df_concat, resultado)

        #salvar métricas
        sr.salvar_resultado(nome, df_matches, dir_metricas)
        sr.salvar_resultado_categoria(nome, df_matches, dir_metricas)

        break

In [ ]:
    a

# Resultados

In [7]:
def print_acu_k(df, flag = True):

    linha_1 = f"  | k-1\t | k-10\t| k-50\t|"

    if flag == True:
        linha_2 = f"1 | {df['k-1'].value_counts()[1]}\t | {df['k-10'].value_counts()[1]}\t| {df['k-50'].value_counts()[1]}\t|"
    else:
        linha_2 = f"1 | {df['k-1'].value_counts()[1]} | {df['k-10'].value_counts()[1]}\t| {df['k-50'].value_counts()[1]}\t|"
    linha_3 = f"0 | {df['k-1'].value_counts()[0]}\t | {df['k-10'].value_counts()[0]}\t| {df['k-50'].value_counts()[0]}\t|"

    print(f"{linha_1}\n{linha_2}\n{linha_3}")


def print_describe(df):

    print("")
    print(df[["k-1", "k-10", "k-50", 'match_rank', '1/match_rank']].describe())

## Geral

In [9]:
arquivos = ['ale_1_1', 'ale_5_1', 'hn_1_1', 'hn_5_1']

lista_df_resultado = []
for arquivo in arquivos:

    df_treino = pd.read_csv(f"Dados/Resultados/Ranqueado/Cos/{arquivo}_métricas.csv")
    lista_df_resultado.append(df_treino)

### Aleátorio

#### 1:1

In [10]:
df_ale_1_1 = lista_df_resultado[0]
print_acu_k(df_ale_1_1)
print_describe(df_ale_1_1)

  | k-1	 | k-10	| k-50	|
1 | 200	 | 367	| 438	|
0 | 258	 | 91	| 20	|

              k-1       k-10        k-50  match_rank  1/match_rank
count  458.000000  458.00000  458.000000  458.000000    458.000000
mean     0.436681    0.80131    0.956332   11.093886      0.556268
std      0.496517    0.39945    0.204579   29.148565      0.412447
min      0.000000    0.00000    0.000000    1.000000      0.003704
25%      0.000000    1.00000    1.000000    1.000000      0.142857
50%      0.000000    1.00000    1.000000    2.000000      0.500000
75%      1.000000    1.00000    1.000000    7.000000      1.000000
max      1.000000    1.00000    1.000000  270.000000      1.000000


#### 5:1

In [11]:
df_ale_5_1 = lista_df_resultado[1]
print_acu_k(df_ale_5_1, False)
print_describe(df_ale_5_1)

  | k-1	 | k-10	| k-50	|
1 | 225 | 474	| 581	|
0 | 406	 | 157	| 50	|

              k-1        k-10        k-50  match_rank  1/match_rank
count  631.000000  631.000000  631.000000  631.000000    631.000000
mean     0.356577    0.751189    0.920761   15.705230      0.485451
std      0.479368    0.432667    0.270326   38.368124      0.407919
min      0.000000    0.000000    0.000000    1.000000      0.002283
25%      0.000000    1.000000    1.000000    1.000000      0.100000
50%      0.000000    1.000000    1.000000    3.000000      0.333333
75%      1.000000    1.000000    1.000000   10.000000      1.000000
max      1.000000    1.000000    1.000000  438.000000      1.000000


### Hard Negative

#### 1:1

In [12]:
df_hn_1_1 = lista_df_resultado[2]
print_acu_k(df_hn_1_1)
print_describe(df_hn_1_1)

  | k-1	 | k-10	| k-50	|
1 | 192	 | 422	| 499	|
0 | 338	 | 108	| 31	|

              k-1        k-10        k-50  match_rank  1/match_rank
count  530.000000  530.000000  530.000000  530.000000    530.000000
mean     0.362264    0.796226    0.941509   11.864151      0.504844
std      0.481109    0.403183    0.234890   32.191835      0.398234
min      0.000000    0.000000    0.000000    1.000000      0.002625
25%      0.000000    1.000000    1.000000    1.000000      0.125000
50%      0.000000    1.000000    1.000000    3.000000      0.333333
75%      1.000000    1.000000    1.000000    8.000000      1.000000
max      1.000000    1.000000    1.000000  381.000000      1.000000


#### 5:1

In [13]:
df_hn_5_1 = lista_df_resultado[3]
print_acu_k(df_hn_5_1, False)
print_describe(df_hn_5_1)

  | k-1	 | k-10	| k-50	|
1 | 266 | 574	| 694	|
0 | 490	 | 182	| 62	|

              k-1        k-10        k-50  match_rank  1/match_rank
count  756.000000  756.000000  756.000000  756.000000    756.000000
mean     0.351852    0.759259    0.917989   16.727513      0.483830
std      0.477864    0.427816    0.274562   47.739468      0.405226
min      0.000000    0.000000    0.000000    1.000000      0.001645
25%      0.000000    1.000000    1.000000    1.000000      0.100000
50%      0.000000    1.000000    1.000000    3.000000      0.333333
75%      1.000000    1.000000    1.000000   10.000000      1.000000
max      1.000000    1.000000    1.000000  608.000000      1.000000


## Por Categoria

### Celulares

In [ ]:
arquivos = ['ale_1_1', 'ale_5_1', 'hn_1_1', 'hn_5_1']

lista_df_celulares = []
for arquivo in arquivos:

    df_celulares = pd.read_csv(f"Dados/Resultados/Cos_Rankeado/celulares/{arquivo}_métricas.csv")
    lista_df_celulares.append(df_celulares)

In [ ]:
df_ale_1_1_cel = lista_df_celulares[0]
df_ale_5_1_cel = lista_df_celulares[1]
df_hn_1_1_cel = lista_df_celulares[2]
df_hn_5_1_cel = lista_df_celulares[3]

In [ ]:
print_acu_k(df_ale_1_1_cel)
print_describe(df_ale_1_1_cel)

In [ ]:
print_acu_k(df_ale_5_1_cel)
print_describe(df_ale_5_1_cel)

In [ ]:
print_acu_k(df_hn_1_1_cel)
print_describe(df_hn_1_1_cel)

In [ ]:
print(f"  | k-1\t | k-10\t| k-50\t|\n1 | {df_hn_5_1_cel['k-1'].value_counts()[1]} | {df_hn_5_1_cel['k-10'].value_counts()[1]}\t| {df_hn_5_1_cel['k-50'].value_counts()[1]}\t|\n0 | {df_hn_5_1_cel['k-1'].value_counts()[0]}\t | {df_hn_5_1_cel['k-10'].value_counts()[0]}\t|\t|")
print_describe(df_hn_5_1_cel)

### Fogões

In [ ]:
arquivos = ['ale_1_1', 'ale_5_1', 'hn_1_1', 'hn_5_1']

lista_df_fogoes = []
for arquivo in arquivos:

    df_fogoes = pd.read_csv(f"Dados/Resultados/Cos_Rankeado/fogoes/{arquivo}_métricas.csv")
    lista_df_fogoes.append(df_fogoes)

In [ ]:
df_ale_1_1_fg = lista_df_fogoes[0]
df_ale_5_1_fg = lista_df_fogoes[1]
df_hn_1_1_fg = lista_df_fogoes[2]
df_hn_5_1_fg = lista_df_fogoes[3]

In [ ]:
print_acu_k(df_ale_1_1_fg)
print_describe(df_ale_1_1_fg)

In [ ]:
print_acu_k(df_ale_5_1_fg)
print_describe(df_ale_5_1_fg)

In [ ]:
print_acu_k(df_hn_1_1_fg)
print_describe(df_hn_1_1_fg)

In [ ]:
print(f"  | k-1\t | k-10\t| k-50\t|\n1 | {df_hn_5_1_fg['k-1'].value_counts()[1]}  | {df_hn_5_1_fg['k-10'].value_counts()[1]}\t| {df_hn_5_1_fg['k-50'].value_counts()[1]}\t|\n0 | {df_hn_5_1_fg['k-1'].value_counts()[0]}\t | {df_hn_5_1_fg['k-10'].value_counts()[0]}\t|\t|")
print_describe(df_hn_5_1_fg)

### Geladeiras

In [ ]:
arquivos = ['ale_1_1', 'ale_5_1', 'hn_1_1', 'hn_5_1']

lista_df_gel = []
for arquivo in arquivos:

    df_gel = pd.read_csv(f"Dados/Resultados/Cos_Rankeado/geladeiras/{arquivo}_métricas.csv")
    lista_df_gel.append(df_gel)

In [ ]:
df_ale_1_1_gel = lista_df_gel[0]
df_ale_5_1_gel = lista_df_gel[1]
df_hn_1_1_gel = lista_df_gel[2]
df_hn_5_1_gel = lista_df_gel[3]

In [ ]:
print_acu_k(df_ale_1_1_gel)
print_describe(df_ale_1_1_gel)

In [ ]:
print_acu_k(df_ale_5_1_gel)
print_describe(df_ale_5_1_gel)

In [ ]:
print_acu_k(df_hn_1_1_gel)
print_describe(df_hn_1_1_gel)

In [ ]:
print_acu_k(df_hn_5_1_gel)
print_describe(df_hn_5_1_gel)

### Notebooks

In [ ]:
arquivos = ['ale_1_1', 'ale_5_1', 'hn_1_1', 'hn_5_1']

lista_df_not = []
for arquivo in arquivos:

    df_not = pd.read_csv(f"Dados/Resultados/Cos_Rankeado/notebooks/{arquivo}_métricas.csv")
    lista_df_not.append(df_not)

In [ ]:
df_ale_1_1_not = lista_df_not[0]
df_ale_5_1_not = lista_df_not[1]
df_hn_1_1_not = lista_df_not[2]
df_hn_5_1_not = lista_df_not[3]

In [ ]:
print_acu_k(df_ale_1_1_not)
print_describe(df_ale_1_1_not)

In [ ]:
print_acu_k(df_ale_5_1_not)
print_describe(df_ale_5_1_not)

In [ ]:
print_acu_k(df_hn_1_1_not)
print_describe(df_hn_1_1_not)

In [ ]:
print_acu_k(df_hn_5_1_not)
print_describe(df_hn_5_1_not)

### TVs

In [ ]:
arquivos = ['ale_1_1', 'ale_5_1', 'hn_1_1', 'hn_5_1']

lista_df_tv = []
for arquivo in arquivos:

    df_tv = pd.read_csv(f"Dados/Resultados/Cos_Rankeado/tvs/{arquivo}_métricas.csv")
    lista_df_tv.append(df_tv)

In [ ]:
df_ale_1_1_tv = lista_df_tv[0]
df_ale_5_1_tv = lista_df_tv[1]
df_hn_1_1_tv = lista_df_tv[2]
df_hn_5_1_tv = lista_df_tv[3]

In [ ]:
print(f"  | k-1\t | k-10\t| k-50\t|\n1 | {df_ale_1_1_tv['k-1'].value_counts()[1]}   | {df_ale_1_1_tv['k-10'].value_counts()[1]}\t| {df_ale_1_1_tv['k-50'].value_counts()[1]}\t|\n0 | {df_ale_1_1_tv['k-1'].value_counts()[0]}\t | {df_ale_1_1_tv['k-10'].value_counts()[0]}\t|\t|")
print_describe(df_ale_1_1_tv)

In [ ]:
print_acu_k(df_ale_5_1_tv)
print_describe(df_ale_5_1_tv)

In [ ]:
print_acu_k(df_hn_1_1_tv)
print_describe(df_hn_1_1_tv)

In [ ]:
print_acu_k(df_hn_5_1_tv)
print_describe(df_hn_5_1_tv)